# Scenemixer
Tool for combining two photographs into a composite image (center + surrounding).

In [ ]:
from __future__ import division
import numpy as np
from skimage import io, transform, filters, color
from skimage import img_as_uint, img_as_float
from tools import *
%matplotlib inline

### Settings

In [ ]:
image_folder = "./images/"  # must end with /
# filenames in []
image_in     = ["lapland.jpeg", "sarka.jpeg", "helsinki.jpeg"]  
image_in     = ["MM4.jpg"]  
                                                
image_out    = image_in     # use the same as image_in
new_size     = (1024, 1024) # in pixels (width, height)
radius       = 248          # radius of central/inside area
radius_out   = 2 * radius
ring_width   = 20
ring_colour  = (128, 128, 128)

### New approach
Define image files as list in `images`. For each file `some.jpg`, the block will produce two files `in_some.jpg` and `out_some.jpg` with either inside or outside block.

In [ ]:
# external ring, inside only and outside only
images  = ["MM4.jpg", "sarka.jpeg"] # <= specify list here

# keep as it is
fn = ["%s%s" % (image_folder, i) for i in images]
fn_gray = "%s%s" % (image_folder, "gray.png")

img  = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn]
gray = crop_and_resize_image(img_as_float(io.imread(fn_gray)), new_size, grayscale=True)
_, ring = make_ring_for_image(img_in[0], radius = radius, width50 = ring_width, colour = ring_colour)
_, ring_out = make_ring_for_image(img_in[0], radius = radius_out, width50 = ring_width, colour = ring_colour)

for ii in np.arange(len(img)):
    print "image: %d %s" % (ii, images[ii])
    fni  = "output/in_%s" % images[ii]
    im1  = combine_images(img[ii], gray, radius)
    im = combine_images(im1, gray, radius_out)
    if ring_width == 0:
        imf = im
    else:
        imf = add_ring(im, ring)
        imf = add_ring(imf, ring_out)            
    io.imsave(fni, imf)
    # and out
    fno  = "output/out_%s" % images[ii]
    im1  = combine_images(gray, img[ii], radius)
    im = combine_images(im1, gray, radius_out)
    if ring_width == 0:
        imf = im
    else:
        imf = add_ring(im, ring)
        imf = add_ring(imf, ring_out)            
    io.imsave(fno, imf)
    

### Original approach
With two image lists, producing all possible combinations. Discards original file names, resulting images are called e.g. `001_002.jpeg`.

In [ ]:
# original (without external circle)
fn_in  = ["%s%s" % (image_folder, i) for i in image_in]
fn_out = ["%s%s" % (image_folder, i) for i in image_out]

img_in  = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_in]
img_out = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_out]
_, ring    = make_ring_for_image(img_in[0], radius = radius, width50 = ring_width, colour = ring_colour)

for iin in np.arange(len(image_in)):
    for iout in np.arange(len(image_out)):
        fn  = "output/%03d_%03d.jpeg" % (iin, iout)
        im  = combine_images(img_in[iin], img_out[iout], radius)
        if ring_width == 0:
            imf = im
        else:
            imf = add_ring(im, ring)
        io.imsave(fn, imf)
    

In [ ]:
# same as original, but with external ring
fn_gray = "%s%s" % (image_folder, "gray.png")
fn_in  = ["%s%s" % (image_folder, i) for i in image_in]
fn_out = ["%s%s" % (image_folder, i) for i in image_out]

img_in  = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_in]
img_out = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_out]
gray = crop_and_resize_image(img_as_float(io.imread(fn_gray)), new_size, grayscale=True)
_, ring = make_ring_for_image(img_in[0], radius = radius, width50 = ring_width, colour = ring_colour)
_, ring_out = make_ring_for_image(img_in[0], radius = radius_out, width50 = ring_width, colour = ring_colour)

for iin in np.arange(len(img_in)):
    for iout in np.arange(len(img_out)):
        print "im %d %d" % (iin, iout)
        fn  = "output/%03d_%03d.jpeg" % (iin, iout)
        im1  = combine_images(img_in[iin], img_out[iout], radius)
        im = combine_images(im1, gray, radius_out)
        if ring_width == 0:
            imf = im
        else:
            imf = add_ring(im, ring)
            imf = add_ring(imf, ring_out)            
        io.imsave(fn, imf)
    